#### **Este notebook se ha adaptado al español, y está construido siguiendo el tutorial del canal de Youtube Python Tutorials for Digital Humanities.**

Enlace a Youtube: https://www.youtube.com/watch?v=qLux8NfSEQQ

Enlace a Github: https://github.com/wjbmattingly/fewshot-text.git

En este caso se usó la primera parte de Cronica de una muerte anunciada.

Importamos las librerías

In [ ]:
!pip install -U spacy
!pip install classy-classification
#!python -m spacy download es_core_news_sm

Classy-classification: an easy an intuitive approach to few-shot classification using sentence-transformer or SpaCy models, by Explosion

In [ ]:
import spacy
import classy_classification
#import es_core_news_sm

Cargamos los datos

In [ ]:
data = {}

with open ('/content/data/morir.txt', "r") as m:
    die = m.read().splitlines()

with open ('/content/data/vehiculo.txt', "r") as a:
    vehicle = a.read().splitlines()

data["morir"] = die
data["vehiculo"] = vehicle

data

{'morir': ['Me dijo cuando vine a verla que estaba tan enferma que sentia cercana su muerte.',
  'Hay que estar siempre de parte del muerto.',
  'Acabo de verlos con un cuchillo de matar puercos.',
  'Senti que era yo la que me iba a morir.',
  'Todos lo vieron salir, con cara de cadaver descompuesto, y todos comprendieron que ya sabia que lo iban a matar.'],
 'vehiculo': ['Ya habia formalizado la compra de su camión de carga.',
  'De la velocidad con la que conducia se accidento la semana pasada.',
  'Siempre andaba en bicicleta o en moto para compensar la falta de dinero.',
  'El motor del vehículo se apagó de repente, teniendo las cuatro llantas puestas.',
  'Al novio le regalaron un automovil convertible con su nombre grabado en letras goticas bajo el escudo de la fabrica.']}

Creamos el pipeline con algunos atributos definidos

In [ ]:
nlp = spacy.blank("es")

#nlp = spacy.load("es_core_news_sm")
nlp.add_pipe(
    "text_categorizer", 
    config={
        "data": data, 
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        #"model": "spacy",
        "device": "cpu"
    }
) 

print(nlp("El dia en que lo iban a matar, Santiago Nasar se levanto a las 5.30 de la mañana.")._.cats)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
{'morir': 0.6140407589828139, 'vehiculo': 0.3859592410171861}


Carga un modelo para dividir por oraciones

In [ ]:
sentence_model = spacy.blank("es")

sentence_model.add_pipe("sentencizer")

Carga la novelita del que se extraera la clasificación

In [ ]:
with open ("/content/data/cronica_de_una_muerte_1_parte.txt", "r") as cr:
    novela = cr.read()


Se divide por oraciones

In [ ]:
sentences = sentence_model(novela)

Se clasifican las oraciones de la novela

In [ ]:
final_data = []
for sentence in sentences.sents:
    doc = nlp(sentence.text)
    final_data.append({"oracion": doc.text, "categoria": doc._.cats})

Consultamos las primeras oraciones

In [ ]:
final_data[:7]

[{'categoria': {'morir': 0.5766887651640328, 'vehiculo': 0.42331123483596717},
  'oracion': 'El dia en que lo iban a matar, Santiago Nasar se levanto a las 5.30 de la mañana para esperar el buque en que llegaba el obispo.'},
 {'categoria': {'morir': 0.5450719833799001, 'vehiculo': 0.45492801662009985},
  'oracion': 'Habia soñado que atravesaba un bosque de higuerones donde caia una llovizna tierna, y por un instante fue feliz en el sueño, pero al despertar se sintio por completo salpicado de cagada de pajaros.'},
 {'categoria': {'morir': 0.538333022923825, 'vehiculo': 0.461666977076175},
  'oracion': 'Siempre soñaba con arboles, me dijo Placida Linero, su madre, evocando 27 años despues los pormenores de aquel lunes ingrato.'},
 {'categoria': {'morir': 0.3006379623039023, 'vehiculo': 0.6993620376960975},
  'oracion': 'La semana anterior habia soñado que iba solo en un avion de papel de estaño que volaba sin tropezar por entre los almendros, me dijo.'},
 {'categoria': {'morir': 0.474145

Consultar las oraciones con categoría mayor a 0.8

In [ ]:
for item in final_data:
  if item['categoria']['morir'] > 0.8:
    print(item['oracion'].strip())
    print(item['categoria'])
    print()

El dia en que lo iban a matar, su madre creyo que el se habia equivocado de fecha cuando lo vio vestido de blanco.
{'morir': 0.8023125718192756, 'vehiculo': 0.19768742818072457}

Al contrario, me dijo cuando vine a verla, poco antes de su muerte.
{'morir': 0.8080295918933426, 'vehiculo': 0.19197040810665733}

Victoria Guzman le mostro el cuchillo ensangrentado.
{'morir': 0.8848547796139807, 'vehiculo': 0.11514522038601908}

No obstante, Divina Flor me confeso en una visita posterior, cuando ya su madre habia muerto, que esta no le habia dicho nada a Santiago Nasar porque en el fondo de su alma queria que lo mataran.
{'morir': 0.8176276423341966, 'vehiculo': 0.18237235766580326}

Mi hermana sintio pasar el angel.
{'morir': 0.8698397293026605, 'vehiculo': 0.13016027069733954}

Tanto, que a veces he pensado que Margot ya sabia que lo iban a matar y queria esconderlo en tu casa.
{'morir': 0.8031742962318358, 'vehiculo': 0.19682570376816427}

En realidad, mi hermana Margot era una de las po